В данном ноутбуке рассмотрим Vosk для перевода русского языка из аудио файла в текстовую форму.

Содержание:
1. Подготовка
2. Процесс распознавания
3. Оценка распознавания
4. Вывод

# 1. Подготовка

Проверим какой CPU используется для того, чтобы занести в конечный отчет.

In [1]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [2]:
!pip install -q vosk wave tqdm

In [3]:
from vosk import Model, KaldiRecognizer, SetLogLevel
from tqdm.notebook import tqdm
import wave
import os
import json
import time

In [4]:
!git clone https://github.com/alphacep/vosk-api

fatal: destination path 'vosk-api' already exists and is not an empty directory.


На данном шаге придется пару минут подождать, так как модель весит 1,5Гб

In [5]:
%cd vosk-api/python/example
#Russian lang model
!wget https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
!unzip vosk-model-ru-0.22.zip
%mv vosk-model-ru-0.22 model

/content/vosk-api/python/example
--2022-10-29 15:55:03--  https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1559419132 (1.5G) [application/zip]
Saving to: ‘vosk-model-ru-0.22.zip.4’

vosk-model-ru-0.22. 100%[===================>]   1.45G  25.2MB/s    in 60s     

2022-10-29 15:56:04 (24.9 MB/s) - ‘vosk-model-ru-0.22.zip.4’ saved [1559419132/1559419132]

Archive:  vosk-model-ru-0.22.zip
replace vosk-model-ru-0.22/graph/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace vosk-model-ru-0.22/graph/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: vosk-model-ru-0.22/graph/words.txt  
 extracting: vosk-model-ru-0.22/graph/phones/silence.csl  
  inflating: vosk-model-ru-0.22/graph/phones/align_lexicon.txt  
  inflating: v

Скачаем пару тестовых файлов для распознавания

In [6]:
%cd /content/

/content


In [7]:
!gdown 1BOoXkBL1FgrFO_7M_BMKmDduqij16maV

Downloading...
From: https://drive.google.com/uc?id=1BOoXkBL1FgrFO_7M_BMKmDduqij16maV
To: /content/test.wav
100% 334k/334k [00:00<00:00, 98.6MB/s]


In [8]:
!gdown 1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2

Downloading...
From: https://drive.google.com/uc?id=1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2
To: /content/tes2.wav
100% 217k/217k [00:00<00:00, 93.0MB/s]


# 2. Процесс распознавания

In [9]:
def transcript_file(input_file, model_path):
    
    # Check if file exists
    if not os.path.isfile(input_file):
        raise FileNotFoundError(os.path.basename(input_file) + " not found")    
    
    # Check if model path exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(os.path.basename(model_path) + " not found")

    # open audio file
    wf = wave.open(input_file, "rb")
    
    # check if wave file has the right properties
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        raise TypeError("Audio file must be WAV format mono PCM.")
    
    # Initialize model
    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())
    
    # Get file size (to calculate progress bar)
    file_size = os.path.getsize(input_file)
    
    # Run transcription
    pbar = tqdm(total=file_size)

    # To store our results
    transcription = []

    while True:
        start_time = time.time()
        data = wf.readframes(4000) # use buffer of 4000
        pbar.update(len(data))
        if len(data) == 0:
            pbar.set_description("Transcription finished")
            break
        if rec.AcceptWaveform(data):
            # Convert json output to dict
            result_dict = json.loads(rec.Result())
            # Extract text values and append them to transcription list
            transcription.append(result_dict.get("text", ""))

    # Get final bits of audio and flush the pipeline
    final_result = json.loads(rec.FinalResult())
    transcription.append(final_result.get("text", ""))
    
    transcription_text = ' '.join(transcription)
    time_1 = round((time.time() - start_time), 3)
    
    return transcription_text, time_1

In [10]:
transcription_1, time_1 = transcript_file('/content/test.wav', '/content/vosk-api/python/example/model')

  0%|          | 0/333842 [00:00<?, ?it/s]

In [11]:
print(f'Распознанный текст:\n*****\n{transcription_1}\n*****\nВремя затраченное на распознавание:{time_1} секунды')

Распознанный текст:
*****
раз два три проверка связи три два раз как слышно приём
*****
Время затраченное на распознавание:0.713 секунды


In [12]:
transcription_2, time_2 = transcript_file('/content/tes2.wav', '/content/vosk-api/python/example/model')

  0%|          | 0/216722 [00:00<?, ?it/s]

In [13]:
print(f'Распознанный текст:\n*****\n{transcription_2}\n*****\nВремя затраченное на распознавание:{time_2} секунды')

Распознанный текст:
*****
так новая попытка по правилу поговорить быстро и невнятно возможно в этом году ерундой
*****
Время затраченное на распознавание:2.701 секунды


# 3. Оценка распознавания

Для оценки распознавания воспользуемся метрикой WER (Word error rate (WER) is a common metric of the performance of an automatic speech recognition (ASR) system).

0 - идеальное значение

In [14]:
!pip install -q evaluate jiwer

     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 115 kB 28.7 MB/s 
     |████████████████████████████████| 163 kB 62.6 MB/s 
     |████████████████████████████████| 441 kB 61.1 MB/s 
     |████████████████████████████████| 212 kB 52.8 MB/s 
     |████████████████████████████████| 127 kB 64.5 MB/s 
     |████████████████████████████████| 1.4 MB 55.0 MB/s 
     |████████████████████████████████| 2.2 MB 50.7 MB/s 
     |████████████████████████████████| 115 kB 68.5 MB/s 


In [15]:
transcription_2

'так новая попытка по правилу поговорить быстро и невнятно возможно в этом году ерундой'

In [16]:
from evaluate import load
wer = load("wer")
predictions = [transcription_2]
references = ["так новая попытка попробую поговорить быстро и совершенно невнятно посмотрим как она справится с этой ерундой"]
wer_score = wer.compute(predictions=predictions, references=references)
print(wer_score)

0.5625


Естественно метрика получилась не самая лучшая, так как говорил быстро и невнятно. Для первого случая с паузами и четкой дикцией замеры проводить не буду там и так понятно, что распознавание будет идеальным. 

Замеры скорости произвели ранее.

# 4. Вывод

В данном ноутбуке рассмотрели один из самых популярных библиотек для распознавания, как видно она справляется достойно. Сравним с другими моделями/библиотеками и выберем лучшую для использования в боте.